# Python-1, Лекция 10

Лектор: Петров Тимур, Фролов Андрей

## Итераторы и итерируемое

Начнем с базы. Внутри Python есть две вещи: iterator и iterable. В чем разница и что это такое?

Iterable - это объект, над которым можно проводить итерацию. Что такое итерация? По сути это процесс перебора элементов (например, строки/множества/списки - это итерируемые объекты)

А что же тогда такое итератор? А это у нас объект, который занимается процессом итерации. По определению это класс, у которого реализованы методы next и iter (для iterable объекта реализуется только сам iter)


In [ ]:
c = [1, 2, 3, 4]
for i in c: # что здесь происходит? Неявно вызывается iter(c)
# Причем iter() работает для так называемых контейнеров (для всех, у кого есть __getitem__ или __iter__)
    print(i)

1
2
3
4


In [ ]:
iter(c)

In [ ]:
for c in 32: #поэтому по числам не получится, они не итерируемые
    print(c)

TypeError: ignored

In [ ]:
n = iter(c)
print(next(n))
print(next(n))
print(next(n))
print(next(n))
print(next(n))

1
2
3
4


StopIteration: ignored

In [ ]:
l = [1, 2, 3]
next(l) #список iterable, но не итератор

TypeError: ignored

Где можно встретить итераторы? Да на самом деле много где!

Например, функция zip возвращает итератор:

In [ ]:
a = [1, 2, 3]
b = [1, 2, 3]
c = zip(a, b)
print(next(c))
print(next(c))
print(next(c))

(1, 1)
(2, 2)
(3, 3)


А также есть функция enumerate() - она делает нумерацию элементов, что можно впоследствие использовать внутри for:

In [ ]:
k = [4, 5, 6]
k_e = enumerate(k)
print(next(k_e))
print(next(k_e))
print(next(k_e))

(0, 4)
(1, 5)
(2, 6)


In [ ]:
for num, el in enumerate(k):
    print(num, el)

0 4
1 5
2 6


### А как сделать свой собственный итератор?

С итераторами мы уже встречались в itertools - собственно название говорит само за себя, она содержит полезные итераторы.

Что есть итератор? С точки зрения Python - это объект, который обладает dunder методом \_\_next\_\_, который берет и возвращает следующий элемент до самого конца. В тот момент, когда происходит конец, он должен возвращать ошибку StopIteration

Давайте делать простой итератор на строках:

In [8]:
class MyIter:
    def __init__(self, s):
        self.s = s
        self.index = 0

    def __next__(self):
        if self.index >= len(self.s):
            raise StopIteration("Ended")
        else:
            self.index += 1
            return self.s[self.index - 1]

a = "abcdefg"
iter = MyIter(a)
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))
print(next(iter))

a
b
c
d
e
f


In [9]:
print(next(iter))
print(next(iter))

g


StopIteration: Ended

Ура, сделали простой рабочий итератор! Как вы можете заметить, внутри next можно делать что угодно (можно изменять элементы, выводить по условиям и так далее). Но этого недостаточно, потому что давайте попробуем сделать следующее:

In [10]:
a = "abcdefg"
iter = MyIter(a)
for i in iter:
    print(i)

TypeError: 'MyIter' object is not iterable

Опа, вроде как сделали итератор, но он не iterable. Как так?

Чтобы итератор стал полным итератором, нам надо добавить dunder метод \_\_iter\_\_, который и должен возвращать объект (вернее то, что нужно итерировать):

In [11]:
class MyIter:
    def __init__(self, s):
        self.s = s
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.s):
            raise StopIteration("Ended")
        else:
            self.index += 1
            return self.s[self.index - 1]

In [12]:
a = "abcdefg"
iter = MyIter(a)
for i in iter:
    print(i)

a
b
c
d
e
f
g


Зачем нужны итераторы?

* Позволяет перебирать элементы (без итераторов нам бы пришлось делать while, обращаться к каждому элементу). По факту проще синтаксис, не более

* Позволяет экономить память

Про второе - мы как-то говорили про раницу между range и xrange во втором Python (range во втором Питоне просто выдает сразу весь список, а его хранить в памяти надо)

Давайте попробуем сделать свой собственный range:

In [17]:
class MyRange:
    def __init__(self, start=0, end=10, step=1):
        self.start = start
        self.end = end
        self.step = step

    def __iter__(self):
        return self

    def __next__(self):
        if self.start >= self.end:
            raise StopIteration
        else:
            self.start += self.step
            return self.start - self.step

for i in MyRange(1, 3, 0.5):
    print(i)

1.0
1.5
2.0
2.5


## Генераторы

В чем разница между генератором и итератором?

* Итератор - объект, позволяющий пройтись по объектам

* Генератор - объект, позволяющий генерировать значения

Формально генератор - это расширение итератора (за счет того, что можно генерировать данные)

Как распознать генератор? Простым словом yield!

In [18]:
def simple_gen():
    yield "Какапо"
    yield "Кеа"
    yield "Ара"
    yield "Какаду"
    yield "Корелла"

s = simple_gen()
print(next(s))
print(next(s))
print(next(s))
print(next(s))
print(next(s))
print(next(s)) # опа, знакомая нам ошибка


Какапо
Кеа
Ара
Какаду
Корелла


StopIteration: 

In [19]:
for i in simple_gen():
    print(i)

Какапо
Кеа
Ара
Какаду
Корелла


Как работает генератор, что за yield?

Все просто:

Мы вызываем функцию. Когда он доходит до yield, то выдает значение. После этого функция переходит как бы в режим ожидания. Когда мы ее вызываем в следующий раз, он начинает с места, где закончил и продолжает. Как закончить жизнь генератора? Сделать return

In [ ]:
def fibonacci(n):
    a, b, counter = 0, 1, 0
    while True:
        if (counter > n):
            return
        yield a
        a, b = b, a + b
        counter += 1

f = fibonacci(5)
print(f)
for x in f:
    print(x, end=" ")

<generator object fibonacci at 0x7f4327abbe50>
0 1 1 2 3 5 

Ну хорошо, в нашем первом примере как-то это явно неудобно, писать кучу строк... А можно, на самом деле, сделать вот такую штуку:

In [ ]:
def simple_gen():
    yield from ["Какапо", "Кеа", "Ара", "Какаду", "Корелла"] #yield from работает только с iterable объектами

s = simple_gen()
print(next(s))
print(next(s))
print(next(s))
print(next(s))
print(next(s))

Какапо
Кеа
Ара
Какаду
Корелла


Где вы могли видеть генераторы? А вот тут:

In [20]:
(i for i in range(10))

<generator object <genexpr> at 0x7cc55f08bdf0>

Что еще крутого есть в генераторах? Для итераторов мы можем только ходить по значениям и делать с ними что-то (итерироваться). А вот в генераторы мы можем отправлять значения!

В чем прикол: на самом деле yield не только дает значения, но и послыает значения. Если воспринимать генератор как итератор, то разницы никакой, в общем-то, а вот если расширять функции генератора, то не совсем.

По дефолту yield выдает None, а поскольку мы ничего с этим не делаем, то как бы и ок. Но мы можем отправить что-то с помощью send(), и таким образом, модернизировать значения!

In [ ]:
def count(firstval=0, step=1):
    counter = firstval
    while True:
        new_counter_val = yield counter # здесь возвращается либо None, либо результат send
        if new_counter_val is None:
            counter += step
        else:
            counter = new_counter_val[0]
            step = new_counter_val[1]

start_value = 2.1
step_value = 0.3
counter = count(start_value, step_value)
for i in range(10):
    new_value = next(counter)
    print(f"{new_value:2.2f}", end=", ")

print()
print("set current count value to another value:")
counter.send((100.5, 2)) # ооотправляем посылочку
for i in range(10):
    new_value = next(counter)
    print(f"{new_value:2.2f}", end=", ")

2.10, 2.40, 2.70, 3.00, 3.30, 3.60, 3.90, 4.20, 4.50, 4.80, 
set current count value to another value:
102.50, 104.50, 106.50, 108.50, 110.50, 112.50, 114.50, 116.50, 118.50, 120.50, 

А еще можем вкидывать ошибки)))

In [ ]:
def count(firstval=0, step=1):
    counter = firstval
    while True:
        try:
            new_counter_val = yield counter
            if new_counter_val is None:
                counter += step
            else:
                counter = new_counter_val
        except Exception:
            yield (firstval, step, counter)

c = count()
for i in range(6):
    print(next(c))
print("Our state")
state_of_count = c.throw(Exception)
print(state_of_count)
for i in range(3):
    print(next(c))

0
1
2
3
4
5
Our state
(0, 1, 5)
5
6
7


И на всякий случай, с чего начали, к тому и пришли: генераторы тоже можно передавать в качестве аргументов функции, опа, доп обертка

In [ ]:
def firstn(generator, n):
    g = generator()
    for i in range(n):
        yield next(g)

print(list(firstn(simple_gen, 3)))

['Какапо', 'Кеа', 'Ара']


## Попугай дня

![](https://i.pinimg.com/originals/2d/59/dc/2d59dc37ef7d2c3767f075e05ed193a6.jpg)

А это не попугай! Это капибара (или ее еще называют водосвинкой, а в Мексике ее вообще зовут кокосовой собачкой)

Самые крупные грызуны на планете, максимально милейшие, дружелюбные и очень хорошо ладят с людьми. Едят траву, а водосвинками называются, потому что они плавают и в целом любят воду

Какие факты есть про них:

1. Они балдежные и очень спокойные

2. Католическая церковь в XVI веке говорила, что капибары из-за их полуводного образа жизни являются рыбами, а поэтому их можно было есть в пост

3. В Аргентине недавно переселились в богатые районы Буэнос-Айреса (где, естественно, много зелени) и теперь там хозяйствуют (поэтому есть кучами мемов с капибарами-коммунистами). А поскольку естественные враги капибар - это кайманы и пумы, то как-то их и не выселишь

![](https://i.pinimg.com/originals/83/0e/16/830e163003a346352ac0d9ba20203b2e.jpg)